In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
#pulp requiere instalarse primero en la consola: pip install pulp
from pulp import LpMaximize, LpProblem, lpSum, LpVariable

In [2]:
#CARGAR SCORES
score_pl = pd.read_csv("score_pl.csv")
score_pl = score_pl.set_index('dia',drop=True)

In [3]:
#CREAR DATAFRAME QUE ALMACENARÁ VARIABLES --> EN ESTE CASO, ES DEPENDIENTE DE LA DIMENSIÓN DE LOS SCORES
calendario = pd.DataFrame(index=score_pl.index.values, columns=score_pl.columns.values)

In [4]:
# Crear un problema de maximización lineal
problema = LpProblem("Maximizar_Resultados", LpMaximize)

In [5]:
# Crear variables de decisión: para la familia j, el día i, se envía correo (1) o no (0)
for j in calendario.columns.values:
    for i in calendario.index.values:
        calendario.loc[i,j] = LpVariable(f"X_{i}_{j}", lowBound=0)  # Variables de decisión
        #calendario.loc[i,j] = LpVariable(f"X_{i}_{j}", lowBound=0, cat="Integer")  # Problema de decisión entera (sólo puede ser 1 o 0)


In [6]:
# Crear función objetivo
problema += lpSum(score_pl.loc[i, j] * calendario.loc[i, j] for i in calendario.index.values for j in calendario.columns.values), "FuncionObjetivo"


In [7]:
# Restricciones
# Restricción de envío díario: máximo 1 por día
for i in calendario.index.values:
    for j in calendario.columns.values:
        #SE DEBE DETERMINAR QUÉ DÍA DE LA SEMANA ES EL DÍA A ASIGNAR
        if datetime.strptime(i,'%Y-%m-%d').weekday() in (5,6): #FIN DE SEMANA, NO HAY ENVÍOS
            problema += calendario.loc[i,j] <= 0, f"Restricción día {i}, familia {j}"
        else: #SI ES DÍA LABORAL, A LO MÁS HAY UN ENVÍO X FAMILIA
            problema += calendario.loc[i,j] <= 1, f"Restricción día {i}, familia {j}"
    
    #SE DETERMINA PARA CADA DÍA, EL MÁXIMO TOTAL DE ENVÍOS A ASIGNAR
    if datetime.strptime(i,'%Y-%m-%d').weekday() in (5,6): #FIN DE SEMANA, NO HAY ENVÍOS
        problema += lpSum(calendario.loc[i,j] for j in calendario.columns.values) <= 0, f"Restricción Máximos envíos el día {i}"
    else:
        problema += lpSum(calendario.loc[i,j] for j in calendario.columns.values) <= 4, f"Restricción Máximos envíos el día {i}"


In [9]:
# Máximos envíos permitidos por familia al mes
envios_4 = ['0209-HERRAMIENTAS Y MAQUINARIAS','0313-BANOS Y COCINAS','0314-PISOS','0316-ELECTROHOGAR','0417-MUEBLES']
envios_3 = ['0101-MADERA Y TABLEROS','0104-TABIQUERIA/TECHUMBRE/AISLACION','0522-AIRE LIBRE','0523-JARDIN']
envios_2 = ['0103-FIERRO/HIERRO/ACERO','0210-FERRETERIA','0328-PUERTAS/VENTANAS/MOLDURAS']
envios_1 = ['0419-DECORACION','0427-ORGANIZACION']
envios_0 = ['0105-OBRA GRUESA','0206-PLOMERIA / GASFITERIA','0207-ELECTRICIDAD','0208-ACCESORIOS AUTOMOVILES','0211-CASA INTELIGENTE','0312-PINTURA Y ACCESORIOS','0415-ILUMINACION Y VENTILADORES','0418-MENAJE','0420-ASEO']

for j in calendario.columns.values:
    #SI LA FAMILIA j ESTÁ EN EL ARREGLO envios_X, se le asignan X envíos.
    if j in envios_4:
        problema += lpSum(calendario.loc[i,j] for i in calendario.index.values) <= 4, f"Restricción Máximos envíos al mes familia {j}"
    
    if j in envios_3:
        problema += lpSum(calendario.loc[i,j] for i in calendario.index.values) <= 3, f"Restricción Máximos envíos al mes familia {j}"

    if j in envios_2:
        problema += lpSum(calendario.loc[i,j] for i in calendario.index.values) <= 2, f"Restricción Máximos envíos al mes familia {j}"

    if j in envios_1:
        problema += lpSum(calendario.loc[i,j] for i in calendario.index.values) <= 1, f"Restricción Máximos envíos al mes familia {j}"

    if j in envios_0:
        problema += lpSum(calendario.loc[i,j] for i in calendario.index.values) <= 0, f"Restricción Máximos envíos al mes familia {j}"


In [10]:
#SE CREA DATAFRAME CON PARES "n_semana","dia", PARA SABER A QUÉ SEMANA k CORRESPONDE EL DÍA i
semana=pd.DataFrame({'n_semana':list(map(lambda x: datetime.strptime(x,'%Y-%m-%d').isocalendar().week, list(calendario.index.values))),'dia':list(calendario.index.values)})

for k in set(map(lambda x: datetime.strptime(x,'%Y-%m-%d').isocalendar().week, list(calendario.index.values))):
    for j in calendario.columns.values:
        #PARA CADA FAMILIA, LA SUMA TOTAL DE ENVÍOS A LA SEMANA NO PUEDE SER MAYOR A 1
        problema += lpSum(calendario.loc[i,j] for i in (semana[semana['n_semana']==k]['dia'])) <= 1, f"Restricción Máximos semana {k} familia {j}"


In [11]:
problema.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /var/tmp/4f231dd524bc412e9745fd6ff40fae66-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/tmp/4f231dd524bc412e9745fd6ff40fae66-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 863 COLUMNS
At line 4312 RHS
At line 5171 BOUNDS
At line 5172 ENDATA
Problem MODEL has 858 rows, 690 columns and 2760 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 106 (-752) rows, 308 (-382) columns and 924 (-1836) elements
0  Obj -0 Dual inf 2.8196734e+09 (308)
49  Obj 6.7323148e+08 Primal inf 13.999986 (14)
65  Obj 6.5543598e+08
Optimal - objective value 6.5543598e+08
After Postsolve, objective 6.5543598e+08, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 655435984 - 65 iterations time 0.002, Presolve 0.00
Option for printi

1

In [21]:
df_final=[]

#SE CREA LISTA PARA PARES DÍA i Y FAMILIA j QUE TUVIERON ASIGNACIÓN
for i in calendario.index.values:
    for j in calendario.columns.values:
        if calendario.loc[i,j].value() > 0:
            df_final.append(i+"_"+j)


In [24]:
#SE EXPORTA LISTA A CSV
pd.DataFrame(df_final).to_csv("calendario.csv")
